In [1]:
import nltk
import pandas as pd
import io
import re
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import random
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('PorterStemmer')

In [2]:
data = pd.read_csv('cleanerstill.csv', sep=";")
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,body type,diet,drinks,drugs,education,essay0,essay4,essay6,essay7,...,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college university,about me i would love to think that i was so...,books absurdistan the republic of mice and ...,duality and humorous things,trying to find someone to hang out with i am ...,...,south san francisco california,doesnt have kids but might want them,straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef this is what that means 1 i am ...,i am die hard christopher moore fan i don't r...,,,...,oakland california,doesnt have kids but might want them,straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,english fluently spanish poorly french ...,single
2,38,thin,anything,socially,,graduated from masters program,i'm not ashamed of much but writing public te...,okay this is where the cultural matrix gets so...,,viewing listening dancing talking drinking...,...,san francisco california,,straight,has cats,,m,pisces but it doesnt matter,no,english french c,available
3,23,thin,vegetarian,socially,,working on college university,i work in a library and go to school,bataille celine beckett lynch jarmusch...,cats and german philosophy,,...,berkeley california,doesnt want kids,straight,likes cats,,m,pisces,no,english german poorly,single
4,29,athletic,,socially,never,graduated from college university,hey how's it going currently vague on the prof...,music bands rappers musicians at the moment...,,,...,san francisco california,,straight,likes dogs and likes cats,,m,aquarius,no,english,single


In [3]:
data['body type'].value_counts()

average           14652
fit               12711
athletic          11819
                   5296
thin               4711
curvy              3924
a little extra     2629
skinny             1777
full figured       1009
overweight          444
jacked              421
used up             355
rather not say      198
Name: body type, dtype: int64

In [4]:
essay0 = [essay for essay in data["essay0"][:30000]]
words = []
all_words = []
essay_list = []
essay_lister = []
stop_words = stopwords.words('english')
porter = PorterStemmer()


type_bod= ["average","fit", "athletic", "curvy", "else"]

def body_t(type_bod,body):
    if "average" == body:
        return "average"
    elif "fit" == body:
        return "fit"
    elif "athletic" == body:
        return "athletic"
    elif "curvy" == body:
        return "curvy"
    else:
        return "else"


for i, essay in enumerate(essay0):
    tmp = []
    tmp_list = []
    if type(essay) != float:
        splt = essay.split()
        words.extend(splt)
        tmp.extend([porter.stem(w) for w in splt if not w in stop_words])
        for w in tmp:
            if "'" in w:
                tmp_list.extend(w.split("'"))
            else:
                tmp_list.append(w)
        essay_list.append((body_t(type_bod,(data["body type"][i])),[word for word in tmp_list if word])) # set identifier

for word in words:
    if "'" in word:
        all_words.extend([w for w in word.split("'") if not w in stop_words])
    else:
        all_words.append(word)

In [5]:
clean_words = [w for w in all_words if not w in stop_words]
stemmed_words = [porter.stem(word) for word in clean_words]
freq_words = nltk.FreqDist(w for w in stemmed_words)
word_features = list(freq_words)[:2000]

In [6]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(t), f) for (f, t) in essay_list if t]
random.shuffle(featuresets)
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [7]:
print(nltk.classify.accuracy(classifier, test_set))

0.304


In [8]:
classifier.show_most_informative_features(10)

Most Informative Features
         contains(curvi) = True            curvy : athlet =     49.4 : 1.0
        contains(dramat) = True            curvy : athlet =     13.7 : 1.0
          contains(riot) = True            curvy : fit    =     12.4 : 1.0
          contains(mesh) = True            curvy : fit    =     12.4 : 1.0
         contains(japan) = True           averag : curvy  =     11.6 : 1.0
         contains(trunk) = True            curvy : else   =     10.2 : 1.0
         contains(simon) = True            curvy : fit    =     10.1 : 1.0
        contains(walker) = True            curvy : athlet =      9.5 : 1.0
         contains(sauci) = True            curvy : athlet =      9.5 : 1.0
          contains(duke) = True            curvy : averag =      8.8 : 1.0
